In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from store.operations import StoringPool
from bidding.operations import create_bidding_app, setup_bidding_app, place_bid, cancel_bid, accept_bid, close_bidding
from trading.operations import create_trading_app, setup_trading_app, place_trade, cancel_trade, accept_trade, close_trading
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))
# close_bidding(client, 693, storeAppID, creator)
print(f"storeAppID: {storeAppID}")

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

storeAppID: 2185
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeApp = StoringPool(client=client, creator=creator)
storeApp.create_app()
storeAppID = storeApp.app_id

# storeAppID = int(os.environ.get("STORE_APP_ID"))
# storeApp.app_id = storeAppID
# print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Waiting for confirmation...
Transaction FOF4EKK2F2J6HXO7LD7PKCHEBOXHMZXSFCFLGCVLCUMUKT3EXS6Q confirmed in round 111902.
Store App ID: 2397
Store App address: EX2UHJY5PVW2KEJIC5JGQXGSJMRWAKEV7ZDUVXC2ETNM5EUWKSFQN5SBNQ
Waiting for confirmation...
Transaction GHIR4JEPPWP336TLECMVG3YZWUC2HBUJOWVEIM77X2RRAUIRBICA confirmed in round 111904.
Store App Address: EX2UHJY5PVW2KEJIC5JGQXGSJMRWAKEV7ZDUVXC2ETNM5EUWKSFQN5SBNQ


In [7]:

appID = create_bidding_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction 2IJRBWEBNRDG6HNNBDZMXSLUM7K3CC4HLJH2NUUT2BNRKQFHSCHA confirmed in round 111906.
Waiting for confirmation...
Transaction HE7V2TRPJZXAC7KMOEVMN6ONA2YQOKK7AGHPTPMN4MWLWTK5JC5A confirmed in round 111908.
App ID: 2399
App Address: GPTPEXTYRVXI5W2IXVIDXMKPV5BH4PXPRUUTJDUFNPR3WCQ2SGKCQMVHVE


Alice is setting up and funding token auction...

In [8]:
tradingAppID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction DH3T4AW6XYXWBSW3FOSFQJQDVQBXSDQQ3J776CB6IGRE3FALTBKQ confirmed in round 111910.
Waiting for confirmation...
Transaction Y7A2BZSKYGQD2YM5HWLMAYSN54CK2QZ6MFYIGU3FUI6W5LVMFSWQ confirmed in round 111912.
App ID: 2399
App Address: GPTPEXTYRVXI5W2IXVIDXMKPV5BH4PXPRUUTJDUFNPR3WCQ2SGKCQMVHVE


In [9]:
storeApp.set_up(trade_app_id=tradingAppID, bid_app_id=appID, auction_app_id=123)

Waiting for confirmation...
Transaction EFIA62ID7LNGLPRRYAS2CGNEHJKNB3ZEB33T5EMJOPPHXK6JTQIQ confirmed in round 111914.


In [10]:
sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Alice's algo balance:  20054676250584  algos


In [11]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 100000}


In [12]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10025193844987  algos
Carla is placing bid for:  1000000  algos


In [13]:
bid_index = place_bid(client=client, app_id=appID, token_id=asset_id, bidder=bidder, bid_amount=10, bid_price=bidPrice, bid_index="")

print("Carla bid", asset_id)

store_app_id 2397
bidder X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU opt in app 2397
Waiting for confirmation...
Transaction Q5T2A5MSYQY4CYAICHJU42QGIWP36CNZYJYHEBBCZYNHD55GAX6Q confirmed in round 111917.
new address: L2TX36NVBXWFYRFCD6DH5KCVDTSATNEFJ6CL7JJEOIP3DSA6KQ5RZQZ7NA
new private_key: eJ39pMJGfoGijeUm+xR1T9QClJyFdeLi/5U/rAhqBUZep335tQ3sXESiH4Z+qFUc5Am0hU+Ev6Ukch+xyB5UOw==
new passphrase: pyramid wait fame history wish park home tortoise gospel fault ritual eager aisle celery coconut intact fox zebra slab sentence bacon fold coral able isolate
Waiting for confirmation...
Transaction 4WJA2S72TSBCUZTTUND3VFV3QRWGGZVNRIFQWGVUSBX74TDGG5ZQ confirmed in round 111919.
Waiting for confirmation...
Transaction ZJ4JVQEAIOZGK7AHWWOWKITNHH3WYQHOPZP3XC3W6SEZYS7QMTDQ confirmed in round 111921.
Waiting for confirmation...
Transaction E6USU7G5NG52L75LNONLOOKP3TEHLFKRMKTE72OJQZTZ3KD46BJA confirmed in round 111923.
obj {'LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I': {

In [14]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 1104000}


In [15]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

# cancel_bid(client, appID, bidder, bid_index)

# print("Carla canceled bid", asset_id)

In [16]:
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=10, bid_price=bidPrice, bid_index="")
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=20, bid_price=bidPrice+1_000_000, bid_index=bid_index)
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=15, bid_price=bidPrice+500_000, bid_index=bid_index)
# print(bid_index)


In [17]:
bidderAppLocalState = get_app_local_state(client, appID, bid_index)
print(bidderAppLocalState)

{b'TA': 10, b'B_ADDR': b'\xbf\xde\xc1=\xf1\x1e\xa7\xbe\xe71\x9b\xc9<+\xbb7q`\xaa\x8dsj<wVp\xdf\xb6\xef\x0c\xfbL', b'TP': 1000000, b'TK_ID': 499}


In [18]:
accept_bid(client, appID, seller, bidder.get_address(), bid_index)

token_amount 10
price 1000000
Waiting for confirmation...
Transaction MVCPZI7TK7BJKU2VXPXEB2UWUTDOM3V4DIRBSOH5TYLVF4RDAM7A confirmed in round 111927.
Waiting for confirmation...
Transaction YYJG7TZYUDPIELIESWUK22ATKV6TQD5P5YUXYTKPXFQSWJ7AHDFA confirmed in round 111929.
Waiting for confirmation...
Transaction WXNX3LSC64OVLRWIC5NACAUHPNQTEIUTIS4ARDHWFY5EF3ZH6BUA confirmed in round 111931.


True

Alice is closing out the bidding contract....

In [19]:
sleep(5)

close_bidding(client, appID, creator, [asset_id])

b'assets' [499]
b'accounts' ['4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU']
Waiting for confirmation...
Transaction HTKCWEHJ5T4CN7X4FDZPJYXGWJLWESFHXSGGM2OB542ENFYGWZWQ confirmed in round 111934.


In [20]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 335500, 499: 0}


In [21]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 135  for token ID:  499


AssertionError: 

In [ ]:
storeAppState = get_app_global_state(client, 1676)
print(storeAppState)
#print(encoding.encode_address(storeAppState[b"C"]))